<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2002/02_Ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El Organismo Supervisor de la Inversión en Energía y Minería es una institución pública adscrita a la Presidencia del Consejo de Ministros del Perú y está encargada de supervisar que las empresas eléctricas, las de combustibles y las del régimen general de Minería del Perú cumplan las normas legales de las actividades que desarrollan.

En [Plataforma Nacional de Datos Abiertos](https://www.datosabiertos.gob.pe/dataset/registro-de-hidrocarburos-l%C3%ADquidos-diciembre-2023-osinergmin/resource/7b1c501c-00f4-4451
) se tiene disponible el registro de grifos y estaciones de servicios, actualizado al 22 de diciembre del 2023.


1. Leer el archivo de datos y e indicar cuál es el tipo de dato que contiene cada atributo.

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
datos = pd.read_csv('Grifos y Estaciones de Servicios_Dic2023.csv', sep = ";")
datos

In [ ]:
datos.info()

2. Seleccione los establecimientos que son de tipo "ESTACIONES DE SERVICIOS" cuya fecha de emisión de registro se encuentre entre el 10 de febrero del 2022 y el 15 de mayo del 2023. Una vez realizado el filtro, seleccionar las columnas RUC, RAZON SOCIAL, TIPO DE ESTABLECIMIENTO, FEC. DE EMISION Y REPRESENTANTE, ordenadas de la fecha más reciente a la antigua, y exportar el data frame resultante en datos2.csv

In [ ]:
datos[['TIPO DE ESTABLECIMIENTO']].value_counts()

In [ ]:
datos['FECHA'] = pd.to_datetime(datos['FEC. EMISION'], format = '%d/%m/%Y')
datos[['FECHA','FEC. EMISION']].head()

In [ ]:
fecha_inicio = pd.to_datetime('2021-02-10')
fecha_fin    = pd.to_datetime('2023-05-15')
datos2       = datos.loc[(datos['TIPO DE ESTABLECIMIENTO'] == "ESTACIONES DE SERVICIOS") & (datos['FECHA'] >= fecha_inicio) & (datos['FECHA'] <= fecha_fin)]
datos2       = datos2[['RUC','RAZON SOCIAL','TIPO DE ESTABLECIMIENTO','FECHA','REPRESENTANTE']].sort_values(by='FECHA', ascending=False).reset_index()
datos2.to_csv('datos2.csv', index=False)

3. Seleccionar los tipos de establecimiento que son PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS que se hayan registrado a inicio o a fin de mes del año 2023. Luego del filtro, seleccionar los atributos RUC, RAZON SOCIAL, DEPARTAMENTO, PROVINCIA, DISTRITO y una columna indicando si el registro se hizo a inicio o fin de mes, y exportar el data frame resultante como datos3.xlsx

In [ ]:
import numpy as np
datos['ANHO']       = datos['FECHA'].dt.year
datos['INICIA_MES'] = datos['FECHA'].dt.is_month_start
datos['FIN_MES']    = datos['FECHA'].dt.is_month_end
datos['MOMENTO']    = np.where(datos['INICIA_MES'] == True, 'INICIO DE MES','FIN DE MES')
datos3 = datos.loc[
    (datos['TIPO DE ESTABLECIMIENTO'] == "PUESTO DE VENTA DE COMBUSTIBLE - GRIFOS") &
     (datos['ANHO'] == 2023) &
      ((datos['INICIA_MES'] == True) | (datos['FIN_MES'] == True))]
datos3 = datos3[['RUC','RAZON SOCIAL','DEPARTAMENTO','PROVINCIA','DISTRITO','MOMENTO']]
datos3.to_excel('datos3.xlsx', index=False)

In [ ]:
datos['INICIA_TRIM']  = datos['FECHA'].dt.is_quarter_start
datos['FIN_TRIM']     = datos['FECHA'].dt.is_quarter_end
datos['INICIA_ANHO']  = datos['FECHA'].dt.is_year_start
datos['FIN_ANHO']     = datos['FECHA'].dt.is_year_end

4. Obtener la capacidad total promedio de combustibles líquidos de los registros realizados los días lunes y que se encuentren dentro de los 5 primeros días del mes.

In [ ]:
datos['DIA_NOMBRE'] = datos['FECHA'].dt.day_name()
datos['DIA_SEMANA'] = datos['FECHA'].dt.weekday + 1
datos['DIA_MES']    = datos['FECHA'].dt.day
datos4              = datos.loc[(datos['DIA_NOMBRE']=="Monday") & (datos['DIA_MES']<=5)]
datos4['CAP.TOTAL CL (gln)'].mean()

In [ ]:
datos.loc[(datos['FECHA'].dt.day_name() == "Monday") & (datos['FECHA'].dt.day <= 5)]['CAP.TOTAL CL (gln)'].mean()

In [ ]:
datos.loc[(datos['FECHA'].dt.weekday == 0) & (datos['FECHA'].dt.day <= 5)]['CAP.TOTAL CL (gln)'].mean()

5. Seleccionar el RUC, Codigo OSINERGMIN, Provincia y Fecha de registro de las instancias correspondientes a la provincia de Lima o Arequipa, y hayan sido hechos en el último dia de los meses que tienen 31 días.

In [ ]:
datos['DIAS_MES']   = datos['FECHA'].dt.days_in_month
datos.loc[((datos['PROVINCIA'] == "LIMA") | (datos['PROVINCIA'] == "AREQUIPA")) & (datos['DIAS_MES'] == 31) & (datos['FIN_MES'] == True)][['RUC','CODIGO OSINERGMIN','PROVINCIA','FECHA']]

In [ ]:
datos.loc[(datos['PROVINCIA'].isin(['LIMA','AREQUIPA']))  & (datos['DIAS_MES'] == 31) & (datos['FIN_MES'] == True)][['RUC','CODIGO OSINERGMIN','PROVINCIA','FECHA']]

6. Seleccionar el departamento, provincia, distrito y fecha de los registros correspondientes al segundo semestre del año 2023 y que no sean del departamento de Lima. Luego, obtener e interpretar la moda de departamento, provincia y distrito.

In [ ]:
import math
datos['TRIMESTRE'] =  datos['FECHA'].dt.quarter
datos['SEMESTRE']  =  datos['TRIMESTRE'].apply(lambda x: math.ceil(x/2))
datos6 = datos.loc[(datos['SEMESTRE'] == 2) & (datos['ANHO'] == 2023) & (datos['DEPARTAMENTO'] != "LIMA")][['DEPARTAMENTO','PROVINCIA','DISTRITO','FECHA']]
datos6

In [ ]:
datos6 = datos.loc[(datos['FECHA'].dt.quarter.isin([3,4])) & (datos['ANHO'] == 2023) & (datos['DEPARTAMENTO'] != "LIMA")][['DEPARTAMENTO','PROVINCIA','DISTRITO','FECHA']]
datos6

In [ ]:
datos6.DEPARTAMENTO.value_counts()

In [ ]:
datos6.DEPARTAMENTO.value_counts().idxmax()

In [ ]:
datos6.DEPARTAMENTO.mode()

In [ ]:
import statistics
statistics.mode(datos6["DEPARTAMENTO"].tolist())

In [ ]:
datos6.PROVINCIA.value_counts().idxmax()

In [ ]:
datos6.PROVINCIA.mode()

In [ ]:
statistics.mode(datos6.PROVINCIA.tolist())

In [ ]:
datos6.DISTRITO.value_counts().idxmax()

In [ ]:
datos6.DISTRITO.mode()

In [ ]:
statistics.mode(datos6.DISTRITO.tolist())

7. Seleccionar el tipo de establecimiento y la fecha de los registros que hayan sido realizados en mayo, noviembre, diciembre o enero del año 2021 o 2023.

In [ ]:
datos['MES']  = datos['FECHA'].dt.month
datos.loc[(datos['MES'].isin([5,11,12])) & datos['ANHO'].isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

In [ ]:
datos.loc[(datos['FECHA'].dt.month.isin([5,11,12])) & datos['FECHA'].dt.year.isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

In [ ]:
datos7 = datos.loc[(datos['FECHA'].dt.month_name().isin(['May','November','December'])) & datos['FECHA'].dt.year.isin([2021,2023])][['TIPO DE ESTABLECIMIENTO','FECHA']]

In [ ]:
datos7.FECHA.value_counts()

In [ ]:
datos7.FECHA.value_counts().idxmax() # precaución con el resultado!

In [ ]:
datos7.FECHA.mode()

In [ ]:
statistics.mode(datos7.FECHA.tolist()) # precaución con el resultado!

8. Seleccionar los registros de los 50 primeros días del año 2023 y cuyo término de vigencia no sea indefinido.

In [ ]:
datos['ANHO']     = datos['FECHA'].dt.year
datos['DIA_ANHO'] = datos['FECHA'].dt.day_of_year
datos.loc[(datos['FECHA'].dt.year == 2023) & (datos['FECHA'].dt.day_of_year <= 50) & (datos['TÉRMINO DE VIGENCIA'] != "INDEFINIDO")]['FECHA']

9. Seleccionar los registros que hayan sido realizados al menos 360 días antes de la fecha actual. Luego, obtener el promedio de días de anticipación.

In [ ]:
from datetime import datetime

datos['ANTICIPACION'] = (datetime.now() - datos['FECHA']).dt.days
datos9 = datos[datos['ANTICIPACION'] >= 360]
datos9[['FECHA','ANTICIPACION']]

In [ ]:
datos9.ANTICIPACION.mean()

10. Seleccionar los registros que hayan sido realizados a partir del 28 de julio del 2023, indicando el RUC, DEPARTAMENTO, FECHA y días desde la fecha de registro.

In [ ]:
datos['DIAS_DFP'] = (datos['FECHA'] - datetime(2023, 7, 28)).dt.days
datos.loc[(datos['DIAS_DFP'] > 0)][['RUC','DEPARTAMENTO','FECHA','DIAS_DFP']]

In [ ]:
import datetime

dt = datetime.date(2023, 9, 16)
day_of_week = dt.weekday()
day_of_week